In [1]:
import os
import math
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import tensorflow as tf
from tensorflow import keras
import py
import mylib
import cv2 as cv
import pytesseract
from tqdm import tqdm
from typing import Optional, List, Dict, Set, Tuple
from scml.nlp import strip_punctuation, to_ascii_str

In [2]:
#os.environ["OMP_THREAD_LIMIT"] = "1"
pd.set_option("use_inf_as_na", True)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)

In [3]:
train = pd.read_csv("input/train.csv", engine="c", low_memory=False)
train["target"] = mylib.target_label(train)
train["image_path"] = "input/train_images/" + train["image"]
posting_ids = train["posting_id"].tolist()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34250 entries, 0 to 34249
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   posting_id   34250 non-null  object
 1   image        34250 non-null  object
 2   image_phash  34250 non-null  object
 3   title        34250 non-null  object
 4   label_group  34250 non-null  int64 
 5   target       34250 non-null  object
 6   image_path   34250 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.8+ MB


In [4]:
class ArcMarginProduct(keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

In [5]:
model_dir = "models/eb3_arc_20210502_0000"
m0 = keras.models.load_model(f"{model_dir}/trial_0/fold_0/model.h5", custom_objects={"ArcMarginProduct": ArcMarginProduct})
m0 = keras.models.Model(inputs=m0.input[0], outputs=m0.get_layer("embedding_output").output)
m0.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 300, 300, 3)]     0         
_________________________________________________________________
efficientnetb3 (Functional)  (None, 1536)              10783535  
_________________________________________________________________
layer_normalization_1 (Layer (None, 1536)              3072      
_________________________________________________________________
dense_1 (Dense)              (None, 1536)              2360832   
_________________________________________________________________
embedding_output (LayerNorma (None, 1536)              3072      
Total params: 13,150,511
Trainable params: 2,366,976
Non-trainable params: 10,783,535
_________________________________________________________________


In [6]:
m1 = keras.models.load_model(f"{model_dir}/trial_0/fold_1/model.h5", custom_objects={"ArcMarginProduct": ArcMarginProduct})
m1 = keras.models.Model(inputs=m1.input[0], outputs=m1.get_layer("embedding_output").output)
m1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 300, 300, 3)]     0         
_________________________________________________________________
efficientnetb3 (Functional)  (None, 1536)              10783535  
_________________________________________________________________
layer_normalization_2 (Layer (None, 1536)              3072      
_________________________________________________________________
dense_2 (Dense)              (None, 1536)              2360832   
_________________________________________________________________
embedding_output (LayerNorma (None, 1536)              3072      
Total params: 13,150,511
Trainable params: 2,366,976
Non-trainable params: 10,783,535
_________________________________________________________________


In [7]:
m2 = keras.models.load_model(f"{model_dir}/trial_0/fold_2/model.h5", custom_objects={"ArcMarginProduct": ArcMarginProduct})
m2 = keras.models.Model(inputs=m2.input[0], outputs=m2.get_layer("embedding_output").output)
m2.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 300, 300, 3)]     0         
_________________________________________________________________
efficientnetb3 (Functional)  (None, 1536)              10783535  
_________________________________________________________________
layer_normalization_3 (Layer (None, 1536)              3072      
_________________________________________________________________
dense_3 (Dense)              (None, 1536)              2360832   
_________________________________________________________________
embedding_output (LayerNorma (None, 1536)              3072      
Total params: 13,150,511
Trainable params: 2,366,976
Non-trainable params: 10,783,535
_________________________________________________________________


In [8]:
m3 = keras.models.load_model(f"{model_dir}/trial_0/fold_3/model.h5", custom_objects={"ArcMarginProduct": ArcMarginProduct})
m3 = keras.models.Model(inputs=m3.input[0], outputs=m3.get_layer("embedding_output").output)
m3.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 300, 300, 3)]     0         
_________________________________________________________________
efficientnetb3 (Functional)  (None, 1536)              10783535  
_________________________________________________________________
layer_normalization_4 (Layer (None, 1536)              3072      
_________________________________________________________________
dense_4 (Dense)              (None, 1536)              2360832   
_________________________________________________________________
embedding_output (LayerNorma (None, 1536)              3072      
Total params: 13,150,511
Trainable params: 2,366,976
Non-trainable params: 10,783,535
_________________________________________________________________


In [9]:
m4 = keras.models.load_model(f"{model_dir}/trial_0/fold_4/model.h5", custom_objects={"ArcMarginProduct": ArcMarginProduct})
m4 = keras.models.Model(inputs=m4.input[0], outputs=m4.get_layer("embedding_output").output)
m4.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 300, 300, 3)]     0         
_________________________________________________________________
efficientnetb3 (Functional)  (None, 1536)              10783535  
_________________________________________________________________
layer_normalization_5 (Layer (None, 1536)              3072      
_________________________________________________________________
dense_5 (Dense)              (None, 1536)              2360832   
_________________________________________________________________
embedding_output (LayerNorma (None, 1536)              3072      
Total params: 13,150,511
Trainable params: 2,366,976
Non-trainable params: 10,783,535
_________________________________________________________________


In [10]:
idg = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    data_format="channels_last",
    dtype=np.float32
)
data = idg.flow_from_dataframe(
    dataframe=train,
    x_col="image",
    y_col="label_group",
    directory="input/train_images",
    target_size=(300, 300),
    color_mode="rgb",
    batch_size=1024,
    shuffle=False,
    class_mode="raw",
    interpolation="nearest",
)
y0 = m0.predict(data, verbose=1)
y1 = m1.predict(data, verbose=1)
#y2 = m2.predict(data, verbose=1)
#y3 = m3.predict(data, verbose=1)
#y4 = m4.predict(data, verbose=1)
assert y0.shape == y1.shape #== y2.shape == y3.shape == y4.shape
print(f"y0.shape={y0.shape}")

Found 34250 validated image filenames.
34/34 [==============================] - 11265s 331s/step
y0.shape=(34250, 1536)


In [11]:
res = []
for i in range(len(y0)):
    #a = np.vstack((y0[i], y1[i], y2[i], y3[i], y4[i]))
    a = np.vstack((y0[i], y1[i]))
    m = np.mean(a, axis=0)
    res.append(m)
em = np.array(res, dtype=np.float32)
assert y0.shape == em.shape
print(f"em.shape={em.shape}")

em.shape=(34250, 1536)


In [12]:
%%time
threshold = 0.1
nn = NearestNeighbors(
    n_neighbors=min(49, len(posting_ids) - 1), metric="euclidean"
)
nn.fit(em)
distances, indices = nn.kneighbors()
res: List[List[str]] = [[] for _ in range(len(indices))]
for i in range(len(indices)):
    for j in range(len(indices[0])):
        if distances[i][j] > threshold:
            break
        res[i].append(posting_ids[indices[i][j]])
train["image_matches"] = res

CPU times: user 2min 21s, sys: 1min 21s, total: 3min 42s
Wall time: 1min 8s


th=.25, f1=.586
th=.30, f1=.586
th=.35, f1=.587
th=.40, f1=.583

In [13]:
%%time
train["phash_matches"] = mylib.phash_matches(train, threshold=0.3)

CPU times: user 27.7 s, sys: 14.6 s, total: 42.3 s
Wall time: 42.3 s


In [14]:
%%time
train["title_p"] = train.apply(mylib.preprocess("title"), axis=1)

CPU times: user 32.8 s, sys: 125 ms, total: 33 s
Wall time: 33 s


In [15]:
%%time
st_name = "stsb-distilbert-base"
#st_name = "paraphrase-distilroberta-base-v1"
#st_name = "paraphrase-xlm-r-multilingual-v1"
train["title_matches"] = mylib.sbert_matches(
    model_path=f"pretrained/sentence-transformers/{st_name}",
    sentences=train["title_p"].tolist(),
    posting_ids=posting_ids,
    threshold=0.5
)

CPU times: user 20min 19s, sys: 1min 4s, total: 21min 23s
Wall time: 4min 3s


In [16]:
def erode_dilate(img):
    kernel = np.ones((2, 2), np.uint8)
    img = cv.erode(img, kernel, iterations=1)
    img = cv.dilate(img, kernel, iterations=1)
    return img


def image_to_text(img_path, mode: str, timeout: float, neighbours: int=41, psm: int=3) -> Optional[str]:
    config = f"--psm {psm}"
    s1, s2 = None, None
    img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
    #img = cv.resize(img, None, fx=0.5, fy=0.5, interpolation=cv.INTER_AREA)
    img = cv.medianBlur(img, 3)
    if mode == "binary_inverted" or mode == "binary":
        th = cv.adaptiveThreshold(img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, neighbours, 2)
        th = erode_dilate(th)
        try:
            s1 = pytesseract.image_to_string(th, timeout=timeout, config=config)
        except:
            s1 = None
    if mode == "binary_inverted" or mode == "inverted":
        th = cv.adaptiveThreshold(img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, neighbours, 2)
        th = erode_dilate(th)
        try:
            s2 = pytesseract.image_to_string(th, timeout=timeout, config=config)
        except:
            s2 = None
    if s1 is None and s2 is None:
        return None
    tokens = []
    if s1 is not None:
        s1 = to_ascii_str(s1)
        s1 = strip_punctuation(s1)
        tokens += s1.split()
    if s2 is not None:
        s2 = to_ascii_str(s2)
        s2 = strip_punctuation(s2)
        tokens += s2.split()
    return " ".join(tokens)

In [17]:
OCR = False
if OCR:
    res = []
    n_timeout = 0
    for t in tqdm(train.itertuples()):
        img_path = getattr(t, "image_path")
        s = image_to_text(img_path, mode="inverted", timeout=0.4, neighbours=41, psm=11)
        if s is None:
            s = ""
            n_timeout += 1
        res.append(s)
    print(f"n_timeout={n_timeout}")

In [18]:
if OCR:
    train["itext"] = res
    train["text"] = train["title"] + " " + train["itext"]
    cols = ["text", "itext", "title"]
    train[cols].head()

In [19]:
%%time
if OCR:
    train["text_p"] = train.apply(mylib.preprocess("text"), axis=1)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.5 µs


In [20]:
if OCR:
    st_name = "stsb-distilbert-base"
    #st_name = "paraphrase-distilroberta-base-v1"
    #st_name = "paraphrase-xlm-r-multilingual-v1"
    train["text_matches"] = mylib.sbert_matches(
        model_path=f"pretrained/sentence-transformers/{st_name}",
        sentences=train["text_p"].tolist(),
        posting_ids=posting_ids,
        threshold=0.5
    )

In [21]:
fs = ["phash_matches", "title_matches", "image_matches"]
if OCR:
    fs.append("text_matches")
train["matches"] = train.apply(mylib.combine_as_list(fs), axis=1)
train["f1"] = train.apply(mylib.metric_per_row("matches"), axis=1)
print(f"Combined score={train.f1.mean():.3f}")

Combined score=0.090


In [22]:
res = [
    {
        "score": 0.088,
        "phash_threshold": 0.3,
        "title_threshold": 0.5,
        "image_threshold": 0.5,
        "text_threshold": None,
        "ocr_threshold": None,
        "ocr_timeout": None,
        "ocr_neighbours": None,
        "ocr_psm": None
    },
    {
        "score": 0.674,
        "phash_threshold": 0.3,
        "title_threshold": 0.5,
        "text_threshold": 0.5,
        "image_threshold": 0.5,
        "ocr_threshold": "inverted",
        "ocr_timeout": 0.4,
        "ocr_neighbours": 41,
        "ocr_psm": 11
    },
    {
        "score": 0.674,
        "phash_threshold": 0.3,
        "title_threshold": 0.5,
        "text_threshold": 0.5,
        "image_threshold": 0.5,
        "ocr_threshold": "binary",
        "ocr_timeout": 0.4,
        "ocr_neighbours": 41,
        "ocr_psm": 11
    },
    {
        "score": 0.674,
        "phash_threshold": 0.3,
        "title_threshold": 0.5,
        "image_threshold": 0.5,
        "text_threshold": None,
        "ocr_threshold": None,
        "ocr_timeout": None,
        "ocr_neighbours": None,
        "ocr_psm": None
    }
]
df = pd.DataFrame.from_records(res)
df.sort_values("score", ascending=False, inplace=True, ignore_index=True)
df.T.head(30)

,0,1,2,3
score,0.674,0.674,0.674,0.088
phash_threshold,0.3,0.3,0.3,0.3
title_threshold,0.5,0.5,0.5,0.5
image_threshold,0.5,0.5,0.5,0.5
text_threshold,0.5,0.5,NaN,NaN
ocr_threshold,inverted,binary,None,None
ocr_timeout,0.4,0.4,NaN,NaN
ocr_neighbours,41.0,41.0,NaN,NaN
ocr_psm,11.0,11.0,NaN,NaN


In [23]:
cols = ["f1", "target", "matches"] + fs
train[cols].head(30)

,f1,target,matches,phash_matches,title_matches,image_matches
0,0.075472,"[train_129225211, train_2278313361]","[train_129225211, train_1309289735, train_2366339891, train_243513028, train_200903631, train_2748139905, train_2573648452, train_2454588595, train_1787280150, train_991683036, train_3088297643, train_3041150821, train_1439009721, train_1146285530, train_432509608, train_2278313361, train_2760452222, train_1861408010, train_3293716060, train_3359300733, train_1899884457, train_2159791112, train_3265145891, train_1388515987, train_753533759, train_2328684746, train_32476901, train_1952799344, train_4180620038, train_2360647092, train_3357267393, train_1938566453, train_685214387, train_13672904, train_2954084829, train_2120095754, train_2745874524, train_3435796905, train_3556769241, train_3980754834, train_1310160910, train_3122164478, train_3036907789, train_3668342417, train_1645355366, train_335719894, train_55865111, train_274485277, train_6477270, train_2795921944, train_1340791296]",[],[train_2278313361],"[train_200903631, train_6477270, train_1861408010, train_3435796905, train_2159791112, train_13672904, train_3357267393, train_1952799344, train_3036907789, train_2745874524, train_2454588595, train_3293716060, train_3359300733, train_1787280150, train_432509608, train_3041150821, train_991683036, train_1146285530, train_1340791296, train_335719894, train_753533759, train_2120095754, train_1309289735, train_55865111, train_3122164478, train_3556769241, train_1899884457, train_1938566453, train_1645355366, train_274485277, train_3088297643, train_4180620038, train_1439009721, train_3980754834, train_2366339891, train_2328684746, train_685214387, train_32476901, train_243513028, train_2954084829, train_2748139905, train_2360647092, train_1310160910, train_2573648452, train_1388515987, train_3265145891, train_2795921944, train_2760452222, train_3668342417]"
1,0.075472,"[train_3386243561, train_3423213080]","[train_3133415900, train_2202259592, train_1395269509, train_2323869681, train_1164304703, train_1968568473, train_3586681855, train_2807111137, train_3024467494, train_2147846099, train_1081931791, train_290486421, train_1927662855, train_2403269575, train_755003837, train_3386243561, train_3430807642, train_1378218030, train_3730004566, train_580639524, train_2591684002, train_3281335331, train_3243886803, train_3423213080, train_626830736, train_2371105215, train_3938323603, train_1188715677, train_3544995555, train_690190523, train_3267429805, train_1909423994, train_2838573674, train_3605286262, train_658167853, train_1026884481, train_2994234361, train_910933116, train_4044410214, train_426195351, train_3441550980, train_3548269661, train_2545348563, train_2758967773, train_2509069129, train_1264882101, train_753570045, train_2558647836, train_2522158364, train_900519743, train_632745654]",[],[train_3423213080],"[train_2545348563, train_2147846099, train_3441550980, train_2202259592, train_1081931791, train_910933116, train_580639524, train_1968568473, train_2807111137, train_3243886803, train_2371105215, train_2994234361, train_3544995555, train_2838573674, train_2323869681, train_626830736, train_632745654, train_3024467494, train_2558647836, train_2403269575, train_290486421, train_2758967773, train_3548269661, train_1927662855, train_1188715677, train_1164304703, train_900519743, train_1378218030, train_4044410214, train_690190523, train_3730004566, train_426195351, train_3133415900, train_3586681855, train_1264882101, train_1026884481, train_1395269509, train_753570045, train_2509069129, train_3281335331, train_2591684002, train_658167853, train_3605286262, train_3267429805, train_1909423994, train_755003837, train_2522158364, train_3430807642, train_3938323603]"
2,0.075472,"[train_2288590299, train_3803689425]","[train_63996627, train_3803689425, train_2185892330, train_4139437520, train_2161215884, train_885706109, train_1569846146, train_646522662, train_2166590500, train_2564717706, train_1741830

In [24]:
train.sort_values("f1", ascending=True, inplace=True, ignore_index=True)
train[cols].head()

,f1,target,matches,phash_matches,title_matches,image_matches
0,0.019802,"[train_1671347582, train_1175819799, train_1525859862, train_1261404451, train_1865413913, train_943427380, train_3789517483, train_3171223531, train_623546295, train_3598710032, train_2375162891, train_3071392783, train_2960536609, train_1149371320, train_219504178, train_518547082, train_242506613, train_349984975, train_1848632939, train_3967271817, train_2309045515, train_100918337, train_612230357, train_913153922, train_450257935, train_1639533062, train_2378747866, train_2305367958, train_726415384, train_948110932, train_1041715554, train_149240701, train_4070413770, train_2901880479, train_897619948, train_1935153677, train_4211797822, train_1369435429, train_613832447, train_2624032383, train_2280558444, train_1182906906, train_1191962025, train_3700503493, train_3207964616, train_2529405184, train_4139159388, train_799917659, train_1630060072, train_282632898, train_3240884969]","[train_4223316428, train_2347650205, train_127090228, train_2167669212, train_1990118232, train_4266957336, train_1369435429, train_1146850576, train_3499107485, train_4252770623, train_4077115599, train_4045506704, train_3041237187, train_1398823299, train_2391922308, train_1607881011, train_1450330719, train_2628505961, train_1273780316, train_462541577, train_3759944592, train_728530820, train_3097670231, train_3227882310, train_3062074862, train_914489157, train_27726943, train_3097411763, train_3542363901, train_3680814205, train_1492895690, train_2102271992, train_818478179, train_574540438, train_617293422, train_515373139, train_2016221292, train_2788214893, train_2212043415, train_3375374759, train_3904448127, train_1643513946, train_4208761047, train_2625177498, train_1488886434, train_3647730166, train_461131136, train_2186527082, train_2044224389, train_635864748]",[],[],"[train_1607881011, train_2044224389, train_3680814205, train_2212043415, train_3499107485, train_1488886434, train_3375374759, train_2628505961, train_1990118232, train_515373139, train_27726943, train_2016221292, train_3904448127, train_3097670231, train_2167669212, train_127090228, train_617293422, train_1643513946, train_4208761047, train_2625177498, train_4252770623, train_2186527082, train_3041237187, train_4045506704, train_1398823299, train_4077115599, train_635864748, train_728530820, train_1492895690, train_3227882310, train_2788214893, train_3647730166, train_3097411763, train_1450330719, train_2347650205, train_574540438, train_1273780316, train_3062074862, train_818478179, train_2102271992, train_462541577, train_461131136, train_1146850576, train_3759944592, train_4266957336, train_3542363901, train_2391922308, train_914489157, train_4223316428]"
1,0.019802,"[train_207039286, train_1619338643, train_3721308865, train_1851882944, train_1236075269, train_2855329700, train_1997808959, train_2840508723, train_2729160884, train_2902736490, train_1789512220, train_2172967167, train_3083046401, train_3596545373, train_1673660532, train_675983017, train_799363063, train_2034561720, train_1459295979, train_1473844208, train_2283849789, train_30488954, train_2937433253, train_3851641413, train_3274592078, train_2847244918, train_4286138740, train_3560974982, train_2072061027, train_3134779648, train_3633103106, train_1906102508, train_3869843407, train_4280580992, train_3671554064, train_2683750575, train_1142799664, train_2513892071, train_3980820034, train_3055192895, train_1495753847, train_2266041454, train_2036340727, train_2338443281, train_467196729, train_1061695682, train_2258929039, train_4238545699, train_2160269935, train_1141098045, train_3414293340]","[train_252985147, train_957290611, train_1877277803, train_3608372934, train_1203116238, train_226883072, train_901065118, train_3272596230, train_1381454307, train_3629499796, train_87681822, train_3628020090, train_1824565844, train_1794537377, train_3449179875, train_3284124770, train_3368756977, train_3872408179, train_26